In [3]:
# Dataset
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
from sklearn.datasets import fetch_mldata
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score

mnist = input_data.read_data_sets("MNIST_data/")

mnist_images = mnist.train.images
mnist_labels = mnist.train.labels

n_three, n_five = sum(mnist_labels==3), sum(mnist_labels==5)

X_all = np.vstack([
    mnist_images[mnist_labels==3,:],
    mnist_images[mnist_labels==5,:]
])

y_all = np.array([1]*n_three + [0]*n_five)
# make it more sparse
X_all = X_all * (np.random.uniform(0, 1, X_all.shape) > 0.8)

print('Dataset shape: {}'.format(X_all.shape))
print('Non-zeros rate: {:.05f}'.format(np.mean(X_all != 0)))
print('Classes balance: {:.03f} / {:.03f}'.format(np.mean(y_all==0), np.mean(y_all==1)))

X_tr, X_te, y_tr, y_te = train_test_split(X_all, y_all, random_state=42, test_size=0.3)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Dataset shape: (10625, 784)
Non-zeros rate: 0.04029
Classes balance: 0.469 / 0.531


In [4]:
# Baseline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
for model in [
    LogisticRegression(), 
    RandomForestClassifier(n_jobs=-1, n_estimators=200)]:
    model.fit(X_tr, y_tr)
    predictions = model.predict(X_te)
    acc = accuracy_score(y_te, predictions)
    print('model: {}'.format(model.__str__()))
    print('accuracy: {}'.format(acc))
    print()

/home/work/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


model: LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)
accuracy: 0.8720200752823086

model: RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
accuracy: 0.8823713927227101



In [5]:
# TFFM svmlight-API
# convert data to svmlight format
from sklearn.datasets import dump_svmlight_file
from sklearn.model_selection import train_test_split
fname = './dump_svmlight.txt'
feature_num=X_all.shape[1]
dump_svmlight_file(X_all, y_all, fname)

import utils
X_cid_all, y_cid_all = utils.read_zipped_column_indexed_data_from_svmlight_file(fname)
X_cid_tr, X_cid_te, y_cid_tr, y_cid_te = train_test_split(X_cid_all, y_cid_all, random_state=42, test_size=0.3)

X_ind_tr, X_val_tr, y_cid_tr = utils.convert_to_column_indexed_data(X_cid_tr, y_cid_tr)
X_ind_te, X_val_te, y_cid_te = utils.convert_to_column_indexed_data(X_cid_te, y_cid_te)

X_ind_tr, X_val_tr, y_cid_tr = utils.convert_to_fully_column_indexed_data(
    X_ind_tr, X_val_tr, y_cid_tr, feature_num=feature_num)
X_ind_te, X_val_te, y_cid_te = utils.convert_to_fully_column_indexed_data(
    X_ind_te, X_val_te, y_cid_te, feature_num=feature_num)

In [6]:
# TFFM svmlight-API
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"

import tfdcnclassifier_column_indexed
import imp
tfdcnclassifier_column_indexed = imp.reload(tfdcnclassifier_column_indexed)
for model in [tfdcnclassifier_column_indexed.TFDCNClassifier(
    feature_num=X_all.shape[1], # feature num must set
    field_num=X_all.shape[1], # field num must set
    factor_num=10,
    deep_layer_nodes=[10, 10],
    cross_layer_num=1,
    l2_weight=0.01, learning_rate=1e-2,
    batch_size=10, epoch_num=10, print_step=1000, random_seed=42)]:
    model.fit(np.array(X_ind_tr), np.array(X_val_tr), np.array(y_cid_tr))
    predictions = model.predict(np.array(X_ind_te), np.array(X_val_te))
    acc = accuracy_score(np.array(y_cid_te), predictions)
    print('model: {}'.format(model.__str__()))
    print('train accuracy: {}'.format(accuracy_score(np.array(y_cid_tr), model.predict(np.array(X_ind_tr), np.array(X_val_tr)))))
    print('test accuracy: {}'.format(acc))

tensorboard --logdir=/tmp/tflog --port 8080
epoch: 1, global_step: 1000, loss: 7.309720516204834
epoch: 2, global_step: 2000, loss: 2.74855375289917
epoch: 4, global_step: 3000, loss: 2.140005588531494
epoch: 5, global_step: 4000, loss: 3.6104345321655273
epoch: 6, global_step: 5000, loss: 4.040725231170654
epoch: 8, global_step: 6000, loss: 3.1432952880859375
epoch: 9, global_step: 7000, loss: 1.6553311347961426
model: <tfdcnclassifier_column_indexed.TFDCNClassifier object at 0x7f48aab55f28>
train accuracy: 0.9147505714669893
test accuracy: 0.8726474278544541
